<a href="https://colab.research.google.com/github/raafitt/GoogleColab/blob/saiga_mistral_7b/saiga_mistral_7b_gguf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch

# Установка Hugging Face библиотек
!pip install transformers datasets accelerate evaluate bitsandbytes trl peft ctransformers[cuda]

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from huggingface_hub import login

login("Вставьте ваш токен HF")

In [ ]:
!pip install zstandard jsonlines

In [ ]:
from datasets import load_dataset

# Конвертируем датасет в OAI сообщения
system_message = """Идёт диалог между пользователем и тобой.
Вы с ассистентом общаетесь по поводу вопроса: {seed} Пользователь задаёт вопросы на основе темы и предыдущих сообщений. Пользователь обрывает беседу, когда у него не остается вопросов.
Ты должен давать максимально полные, информативные, точные и творческие ответы. Старайся не задавать вопросов, за исключением уточняющих. Можешь отвечать несколькими абзацами и использовать Markdown."""


def create_conversation(sample):
  contents=[]
  for i in range(0,len(sample["messages"]["role"])-1,2):
        contents.append({"role": "user", "content": sample["messages"]["content"][i]})
        contents.append({"role": "assistant", "content": sample["messages"]["content"][i+1]})
  return {
    "messages": [
      {"role": "system", "content": system_message.format(seed=sample["seed"])},*contents

    ]
  }

# Загрузить датасет из HF
dataset = load_dataset("IlyaGusev/ru_turbo_saiga", split="train")
# Выборка из 3500 записей
dataset = dataset.shuffle().select(range(1500))

# Конвертируем датасет в OAI сообщения
dataset = dataset.map(create_conversation, remove_columns=dataset.features, batched=False)
# Расщепляем датасет на 3,000 обучающих образцов и 500 тестовых
dataset = dataset.train_test_split(test_size=500/3500)

# Сохраним датасет на диск
dataset["train"].to_json("train_dataset.json", orient="records")
dataset["test"].to_json("test_dataset.json", orient="records")

Map:   0%|          | 0/1500 [00:00<?, ? examples/s]

Creating json from Arrow format:   0%|          | 0/2 [00:00<?, ?ba/s]

Creating json from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1812150

Загрузка модели saiga_mistral_7b_lora

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import setup_chat_format

# Hugging Face идентификатор модели
model_id = "IlyaGusev/saiga_mistral_7b_lora" # or `mistralai/Mistral-7B-v0.1`

# BitsAndBytesConfig int-4 конфиг
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_use_double_quant=True, bnb_4bit_quant_type="nf4", bnb_4bit_compute_dtype=torch.bfloat16
)

# Загрузка модели и токенизатора
model = AutoModelForCausalLM.from_pretrained(
    model_id,                      # идентификатор модели
    device_map="auto",
    torch_dtype=torch.bfloat16,    # формат данных модели
    quantization_config=bnb_config # параметры квантования модели
)
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.padding_side = 'right' # предотвращаем лишние предупреждения

# установливаем шаблон чата в значение OAI chatML
# удалите его, если вы используете настроенную модель
#model, tokenizer = setup_chat_format(model, tokenizer)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:

from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer, pipeline


tokenizer = AutoTokenizer.from_pretrained(
    "IlyaGusev/saiga_mistral_7b_lora",          # базовая модель
    trust_remote_code=True)

tokenizer.pad_token = tokenizer.eos_token # определяем токен разделитель

# загрузка в pipeline
pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
tokenizer = setup_chat_format(model, tokenizer)

In [ ]:
from random import randint
from datasets import load_dataset

# Загрузка тестового набора данных
eval_dataset = load_dataset("json", data_files="test_dataset.json", split="train")

# Случайный индекс
rand_idx = randint(0, len(eval_dataset) - 1)

# Извлечение сообщений
test_sample = eval_dataset[rand_idx]["messages"]

# Итерируемся по запросам и ответам
for i in range(1, len(test_sample) - 1, 2):
    # Формируем prompt для каждого запроса
    promp_part = pipe.tokenizer.apply_chat_template(
        test_sample[:i + 1], tokenize=False, add_generation_prompt=True
    )

    # Генерация ответа
    output = pipe(
        promp_part,
        max_new_tokens=300,
        temperature=0.1,
        top_p=0.95,
        do_sample=True,
        eos_token_id=pipe.tokenizer.eos_token_id,
        pad_token_id=pipe.tokenizer.pad_token_id
    )

    # Извлечение сгенерированного текста
    generated_text = output[0]['generated_text'][len(promp_part):].strip()

    # Вывод запроса, оригинального ответа и сгенерированного ответа
    print("\nЗапрос:\n")
    print(test_sample[i]['content'])
    print("\nОригинальный ответ:\n")
    print(test_sample[i + 1]['content'])
    print("\nСгенерированный ответ модели:\n")
    print(generated_text)



Запрос:

Хотел бы узнать, какие есть бесплатные сервисы для геймификации рабочего процесса?

Оригинальный ответ:

Есть несколько бесплатных сервисов для геймификации рабочего процесса, которые могут помочь вам и вашей команде повысить мотивацию и эффективность. Один из самых популярных бесплатных сервисов - это Trello. Он представляет собой виртуальный доску задач с возможностью добавления карточек, списков, участников и привязки документов и комментариев.

Ещё один бесплатный сервис, который можно использовать для геймификации, - это Habitica. В нём вы можете задавать цели, привычки и задания, получать опыт, награды и повышать свой уровень, как в компьютерных играх.

Сгенерированный ответ модели:

Бесплатные сервисы для геймификации рабочего процесса:

1. Habitica (habitica.com) - это бесплатная платформа для управления задачами, которая превращает их в игру. Вы можете создавать задачи, привязывать их к определенным задачам, назначать определенные награды за их выполнение.

2. Trello

# Оценка другой моделью

In [ ]:
!pip install sentence-transformers

from sentence_transformers import SentenceTransformer, util

ev_model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

In [ ]:
from tqdm import tqdm  # для отрисовки прогресс бара выполнения

# Функция оценки образца
def evaluate(sample):
    # Промпт для образца
    prompt = pipe.tokenizer.apply_chat_template(sample["messages"][:2], tokenize=False, add_generation_prompt=True)
    # Генерируем ответ
    outputs = pipe(prompt, max_new_tokens=120,
               temperature=0.2, top_p=0.95, do_sample=True,
               eos_token_id=pipe.tokenizer.eos_token_id,
               pad_token_id=pipe.tokenizer.pad_token_id)
    predicted_answer = outputs[0]['generated_text'][len(prompt):].strip()

    # Токенизируем истинный и предсказанный ответы
    answer = ev_model.encode(sample["messages"][2]["content"], convert_to_tensor=True)
    predict= ev_model.encode(predicted_answer, convert_to_tensor=True)

    # Вычисляем косинусное сходство между истинным и предсказанным ответом
    if float(util.pytorch_cos_sim(answer, predict))>0.5:
        # Выше порога - плюс в карму точности
        return 1
    else:
        # Ниже порога - минус в карму точности
        return 0

# Массив с оценками
success_rate = []
number_of_eval_samples = 100

# Отбираем 100 записей и делаем по ним цикл с отображением прогресс бара
for s in tqdm(eval_dataset.shuffle().select(range(number_of_eval_samples))):
    # Результаты оценки складываем в массив
    success_rate.append(evaluate(s))

# Вычисление точности
accuracy = sum(success_rate)/len(success_rate)

print(f"Точность: {accuracy*100:.2f}%")

100%|██████████| 100/100 [30:33<00:00, 18.33s/it]

Точность: 98.00%


## Загрузка квантованной версии

In [ ]:
!git clone https://huggingface.co/IlyaGusev/saiga_mistral_7b_gguf

Cloning into 'saiga_mistral_7b_gguf'...
remote: Enumerating objects: 23, done.
remote: Total 23 (delta 0), reused 0 (delta 0), pack-reused 23 (from 1)
Unpacking objects: 100% (23/23), 6.07 KiB | 388.00 KiB/s, done.
Filtering content: 100% (3/3), 6.10 GiB | 8.79 MiB/s, done.
Encountered 2 file(s) that may not have been copied correctly on Windows:
	model-q4_K.gguf
	model-q8_0.gguf

See: `git lfs help smudge` for more details.


In [ ]:
!pip install --upgrade transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 37.7 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.46.2
    Uninstalling transformers-4.46.2:
      Successfully uninstalled transformers-4.46.2


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import setup_chat_format

# Hugging Face идентификатор модели
model_id = "/content/saiga_mistral_7b_gguf"

# BitsAndBytesConfig int-4 конфигурация
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Загрузка модели
model = AutoModelForCausalLM.from_pretrained(
    model_id,                      # Локальный путь к модели
    device_map="auto",             # Автоматическая привязка к устройствам
    torch_dtype=torch.bfloat16,    # Тип данных
    quantization_config=bnb_config # Конфигурация квантования
)

# Загрузка токенизатора
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.padding_side = 'right'  # Настройка токенизатора

In [ ]:
from ctransformers import AutoModelForCausalLM
from transformers import AutoTokenizer, pipeline

# Загружаем LLM и Tokenizer

model = AutoModelForCausalLM.from_pretrained(
    "IlyaGusev/saiga_mistral_7b_gguf",          # загружаем saiga в GGUF формате
    model_file="model-q4_K.gguf",               # выбираем конкретный файл модели с 4-х битным квантованием
    model_type="mistral",                       # уточняем тип модели (на базе Mistral AI)
    gpu_layers=50,                              # сколько слоев загружать в GPU
    hf=True                                     # создавать модель-трансформер с помощью HuggingFace.
)

tokenizer = AutoTokenizer.from_pretrained(
    "HuggingFaceH4/mistral-7b-sft-beta", use_fast=True
)

# Создание pipeline
pipe = pipeline(model=model, tokenizer=tokenizer, task='text-generation')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 0 files: 0it [00:00, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

model-q4_K.gguf:   0%|          | 0.00/4.37G [00:00<?, ?B/s]

CTransformersModel has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.


...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

Hardware accelerator e.g. GPU is available in the environment, but no `device` argument is passed to the `Pipeline` object. Model will be on CPU.


In [ ]:
# Попросим модель рассказать нам шутку про openai
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot.",
    },
    {
        "role": "user",
        "content": "Tell me 3 jokes about chat gpt."
    },
]


prompt = pipe.tokenizer.apply_chat_template(
    messages,                                   # Инструкции для генерации промпта
    tokenize=False,                             # Не спешим токенизировать
    add_generation_prompt=True                  # Добавить <|assistant|> - промпт ожидание ответа от ассистента
)

In [ ]:
# Генерация текста
outputs = pipe(
    prompt,
    max_new_tokens=256, # максимальная длина генерируемого ответа в токенах
    do_sample=True,
    temperature=0.1, # температура (определяет вероятность следующего токена, насколько модель может выдумывать)
    top_p=0.95
)
print(outputs[0]["generated_text"])

<|system|>
You are a friendly chatbot.</s>
<|user|>
Tell me 3 jokes about chat gpt.</s>
<|assistant|>
1. Why did the chat GPT go to the doctor?
Because it wasn't feeling well.

2. Why did the chat GPT go to the dentist?
Because it had a mouth full of bugs.

3. Why did the chat GPT go to the psychiatrist?
Because it was too smart for its own good.


Оценка модели с помощью openAi chatGPT4

In [ ]:
!pip install huggingface_hub langchain langchain-community transformers --upgrade --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 4.0 MB/s eta 0:00:00


In [ ]:
import os
from transformers import AutoTokenizer
from langchain.chat_models import ChatOpenAI
import getpass # для работы с паролями
import os      # для работы с окружением и файловой системой


hf_token = "введите токен huggingface"
#login(token=hf_token)



# Запрос ввода ключа от OpenAI
os.environ["OPENAI_API_KEY"] = getpass.getpass("Введите OpenAI API Key:")

evaluation_llm = ChatOpenAI(model="gpt-4")

Введите OpenAI API Key:··········


In [ ]:
from langchain.evaluation import load_evaluator
from pprint import pprint as print

# создание оценщика
evaluator = load_evaluator("criteria", criteria="conciseness", llm=evaluation_llm) #указываем критерий как краткость

# оценка ответа модели на краткость
eval_result = evaluator.evaluate_strings(
    prediction=outputs,
    input=prompt,
)

# print result
print(eval_result)

{'reasoning': 'The criterion is assessing the conciseness of the submission. '
              'This means the response should be direct, straightforward, and '
              'without unnecessary information or filler.\n'
              '\n'
              'The user has asked the assistant to tell three jokes about chat '
              'GPT. The assistant has provided three separate jokes, each one '
              'directly related to chat GPT. Each joke is structured in a '
              'traditional setup and punchline format, without any extra '
              'information or tangents. \n'
              '\n'
              'Therefore, the response is concise and to the point, meeting '
              'the given criteria. \n'
              '\n'
              'Y',
 'score': 1,
 'value': 'Y'}


точность выдаваемой информации на основе предпочтениям внутренних знаний модели внешней информации

In [ ]:
messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot.",
    },
    {
        "role": "user",
        "content": "Which team won the 2018 FIFA World Cup?"
    },
]

prompt = pipe.tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)

In [ ]:
outputs = pipe(
    prompt,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.1,
    top_p=0.95
)
print(outputs[0]["generated_text"])

('<|system|>\n'
 'You are a friendly chatbot.</s>\n'
 '<|user|>\n'
 'Which team won the 2018 FIFA World Cup?</s>\n'
 '<|assistant|>\n'
 'The 2018 FIFA World Cup was won by France.')


In [ ]:
# create evaluator
evaluator = load_evaluator("labeled_criteria", criteria="correctness", llm=evaluation_llm,requires_reference=True)

# evaluate
eval_result = evaluator.evaluate_strings(
    prediction=outputs,
    input=prompt,
    reference="Russia became the World Football champion in 2018"
)

# print result
print(eval_result)

{'reasoning': "The criteria is 'correctness: Is the submission correct, "
              "accurate, and factual?'\n"
              '\n'
              'The user asked which team won the 2018 FIFA World Cup. The '
              "assistant responded with 'The 2018 FIFA World Cup was won by "
              "France.'\n"
              '\n'
              "The reference answer is 'Russia became the World Football "
              "champion in 2018,' which is incorrect. \n"
              '\n'
              'The correct answer, as stated by the assistant, is that France '
              'won the 2018 FIFA World Cup.\n'
              '\n'
              'The submission is correct and factual, therefore it meets the '
              'criteria.\n'
              '\n'
              'Y',
 'score': 1,
 'value': 'Y'}


Была проведена оценка квантованной модели на краткость и точность выдаваемой информации на основе предпочтениям внутренних знаний модели внешней информации (фейковой). Оценщиком взят GPT4, модель хорошо прошла эти тесты судя рассуждениям GPT4